In [1]:
from tqdm.notebook import tqdm
import unicodedata
import random

global folder
folder = "/content/drive/MyDrive/Afstudeerproject/"

# Data processing

##Data download

In [ ]:
import requests

url = "https://staff.fnwi.uva.nl/g.behnke/output.txt"
r = requests.get(url, allow_redirects=True)

with open(folder + 'Data/raw_data.txt', 'wb') as f:
    for chunk in r.iter_content(chunk_size=128):
        f.write(chunk)

## Raw data

In [ ]:
with open(folder + "Data/raw_data.txt", "rb") as f:
    with open(folder + "Data/raw_data_cleaned.txt", "w") as g:
        lines = f.readlines()
        random.shuffle(lines)
        for line in lines:
            try:
                text = line.decode("utf-8")
                text = unicodedata.normalize("NFKD", text)
                g.write(text.split("\t")[1].replace("==> ", ""))
            except:
                continue

print("Amount of citations:", len(lines))

Amount of citations: 3422628


## Data filtering

In [ ]:
with open(folder + "Data/raw_data_cleaned.txt", "r") as f:
    with open(folder + "Data/auflage_data.txt", "w") as g:
        lines = f.readlines()
        fit = 0
        for line in lines:
            if "Aufl" in line:
                g.write(line)
                fit += 1

print("Amount of fitting citations:", fit)

Amount of fitting citations: 173177


##Example creation

In [ ]:
with open(folder + "Data/example-citations.txt", "rb") as f:
    x = f.read()

texts = unicodedata.normalize("NFKD", x.decode("utf-8")).split("\n")
with open(folder + "Data/examples.txt", "w") as f:
    for part in texts:
        if "ECLI" in part:
            f.write((part.split("\t")[1].replace("==> ", "") + "\n"))

# Lists cleaning

## Names cleaning


In [ ]:
with open(folder + "Data/Wordlists/zitate-names-sorted-with-count-actual-atleast-twice.txt", "r") as f:
    with open(folder + "Data/Wordlists/known_names.txt", "w") as n:
        lines = f.readlines()
        for line in lines:
            while line[0] == " ":
                line = line[1:]
            count, token = line.split(" ")
            n.write(unicodedata.normalize("NFKD", token.replace(":", "")))

## Edge Case cleaning

In [ ]:
with open(folder + "Data/Wordlists/zitate-names-sorted-with-count-edge-cases.txt", "r") as f:
    with open(folder + "Data/Wordlists/only_titles.txt", "r+") as g:
        with open(folder + "Data/Wordlists/known_names.txt", "r+") as h:
            lines = f.readlines()
            titles = g.readlines()
            names = h.readlines()
            for line in lines:
                if line.count("-") != 1:
                    continue
                while not line[0].isalpha():
                    line = line[1:]
                if "-" in line:
                    title, name = line.split("-", 1)
                    if title not in titles:
                        print("title:", title)
                        #g.write(title)
                        titles.append(title)
                    if name not in names:
                        #h.write(name)
                        print("name:", name)
                        names.append(name)

## Titles cleaning

In [ ]:
with open(folder + "Data/Wordlists/zitate-names-sorted-with-count-non-names.txt", "r") as f:
    with open(folder + "Data/Wordlists/only_titles.txt", "w") as n:
        lines = f.readlines()
        for line in lines:
            line = unicodedata.normalize("NFKD", line)
            if line == "\n":
                continue
            try:
                while not line[0].isalpha():
                    line = line[1:]
                upper = 0
                for char in line:
                    if char.isupper():
                        upper += 1
                if upper > 1:
                    n.write(line)
            except:
                continue

## Wordlist cleaning

In [ ]:
with open(folder + "Data/Wordlists/wortliste.txt", "r") as f:
    with open(folder + "Data/Wordlists/wordlist.txt", "w") as g:
        lines = f.readlines()
        for line in lines:
            line = unicodedata.normalize("NFKD", line).lower()
            g.write(line)


# Four blocks data + aufl

In [ ]:
with open(folder + "Data/raw_data_cleaned.txt", "r") as f:
    all_citations = f.readlines()
    
with open(folder + "Data/four_block.txt", "w") as f:
    for i in tqdm(range(len(all_citations))):
        citation = all_citations[i]
        if citation.count(", ") == 3 and "Aufl" in citation:
            f.write(citation)

  0%|          | 0/3422627 [00:00<?, ?it/s]

# Three blocks data

In [ ]:
with open(folder + "Data/Wordlists/known_names.txt", "r") as f:
    namelist = f.readlines()
    namelist = [name.replace("\n", "") for name in namelist]

with open(folder + "Data/raw_data_cleaned.txt", "r") as f:
    all_citations = f.readlines()

randnumbers = ["Rn", "Rdnr", "Rdn", "Rz", "RdNr"]

In [ ]:
with open(folder + "Data/three_block.txt", "w") as f:
    fitting = 0
    for i in tqdm(range(len(all_citations))):
        citation = all_citations[i]
        if citation.count(", ") != 2 or citation.count("a.") >= 2 or "aaO" in citation:
            continue

        rand = False
        for randnumber in randnumbers:
            if randnumber in citation:
                rand = True

        if not rand:
            continue
        
        if citation.count("/") > 0:
            citation = citation.replace("/", " ")
        else:
            citation = citation.replace("-", " ")

        tokens = citation.split(",")
        names = tokens[0].split(" ")
        found = False
        for name in namelist:
            if name in tokens:
                fitting += 1
                f.write(citation)
                found = True
                break
             
print(fitting)

  0%|          | 0/3422627 [00:00<?, ?it/s]

2907


# Know name citations

In [ ]:
with open(folder + "Data/Wordlists/zitate-h├ñufige-namen-und-paragraph-uniq.txt", "rb") as f:
    with open(folder + "Data/known_name_citations.txt", "w") as g:
        lines = f.readlines()
        random.shuffle(lines)
        for line in lines:
            try:
                text = line.decode("utf-8")
                text = unicodedata.normalize("NFKD", text)
                if text.count("a.") >= 2 or "aaO" in text:
                    continue
                g.write(text.split("\t")[1].replace("==> ", ""))
            except:
                continue

print("Amount of citations:", len(lines))

Amount of citations: 386970
